In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import IncrementalPCA

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# Path to people.csv from ReadHatKaggle data set
PEOPLE_FILE_PATH='/Users/sanjanajayakumar/Desktop/Predicting Red Hat Business Value/people.csv'
# Path to act_train.csv from RedHatKaggle data set
ACTIVITIES_FILE_PATH='/Users/sanjanajayakumar/Desktop/Predicting Red Hat Business Value/act_train.csv'
# Path to test.csv from RedHatKaggle data set
TEST_DATA_FILE_PATH='/Users/sanjanajayakumar/Desktop/Predicting Red Hat Business Value/act_test.csv'

In [70]:
# Read the data set's
people_df=pd.read_csv(PEOPLE_FILE_PATH,parse_dates=["date"],true_values=["True"],false_values=["False"])
activity_df=pd.read_csv(ACTIVITIES_FILE_PATH, parse_dates=["date"])
test_df=pd.read_csv(TEST_DATA_FILE_PATH,parse_dates=["date"])

In [71]:
# Introduce a category for null values called category 0 since scikit needs numeric data
people_df.fillna("type 0", inplace=True)
activity_df.fillna("type 0", inplace=True)
test_df.fillna("type 0", inplace=True)

In [72]:
# Rename columns under people_df to avoid same-names in the three dataframe
people_df = people_df.rename(columns=lambda x : "people_" + x)
#test_df = test_df.rename(columns=lambda x : "test_" + x)

In [73]:
# Merge activity and test data frame with people_df
train_dataset = pd.merge(people_df, activity_df, how='right', left_on='people_people_id', right_on="people_id")
test_dataset = pd.merge(people_df, test_df, how='right', left_on='people_people_id', right_on="people_id")

In [74]:
# clearing memory
del people_df
del activity_df
del test_df

NameError: name 'activity_df' is not defined

In [75]:
def category_to_label_encoding(dataset,identity_columns=['people_people_id','people_id','activity_id']):
    for column in dataset.columns:
        if column not in identity_columns:
            if (dataset[column].dtype == 'O'):
                dataset[column]=dataset[column].apply(lambda x: str(x).split(' ')[1]).astype(np.int32)
              # This converts bool to int, but scikit treats boolean as int
#           elif dataset[column].dtype == 'bool':
#                 le=LabelEncoder()
#                 le.fit(['True','False'])
#                 dataset[column]=le.transform(dataset[column])
    return dataset

In [76]:
train_dataset = category_to_label_encoding(train_dataset)
test_dataset = category_to_label_encoding(test_dataset)

In [87]:
# train_dataset['people_people_id']=train_dataset['people_people_id'].apply(lambda x: str(x).split('_')[1]).astype(np.float32)
# test_dataset['people_people_id']=test_dataset['people_people_id'].apply(lambda x: str(x).split('_')[1]).astype(np.float32)
test_dataset['people_id']=test_dataset['people_id'].apply(lambda x: str(x).split('_')[1]).astype(np.float32)
# train_dataset['people_id']=train_dataset['people_id'].apply(lambda x: str(x).split('_')[1]).astype(np.float32)
# test_dataset['activity_id_num']=test_dataset['activity_id'].apply(lambda x: str(x).split('_')[1]).astype(np.float32)
# train_dataset['activity_id_num']=train_dataset['activity_id'].apply(lambda x: str(x).split('_')[1]).astype(np.float32)

In [11]:
activity_id = train_dataset["activity_id"]
del train_dataset["activity_id"]
activity_id_test = test_dataset["activity_id"]
del test_dataset["activity_id"]
#train_dataset['activity_id'] = activity_id
#train_dataset
#test_dataset['activity_id'] = activity_id
#test_dataset

In [12]:
del train_dataset['date']
del train_dataset['people_date']
del test_dataset['date']
del test_dataset['people_date']

In [83]:
y=train_dataset.sort_values(by='activity_id_num')['outcome']
NON_FEATURES=['activity_id_num','people_people_id','outcome']
x=train_dataset.sort_values(by='activity_id_num').drop(NON_FEATURES,axis=1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(x.as_matrix(), y.as_matrix(), test_size=0.3, random_state=0)

In [84]:
ipca = IncrementalPCA(n_components=2, batch_size=3)
ipca.fit(x.as_matrix())

IncrementalPCA(batch_size=3, copy=True, n_components=2, whiten=False)

In [1]:
X_new_t = ipca.transform(x.as_matrix())

NameError: name 'ipca' is not defined

In [22]:
X_new

array([[-15574.61458016,   -467.53836909],
       [ -6126.61155099,   -471.29029644],
       [   995.38650064,   -473.35838659],
       ..., 
       [   790.63844145,    156.51488902],
       [  2181.9045631 ,   1227.23620213],
       [-12714.47277474,     18.62673799]])

In [34]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=5)
print (cross_val_score(knn, x.as_matrix(),y.as_matrix(),cv=10))

KeyboardInterrupt: 

In [31]:
X_test_new=ipca.transform(X_test)

In [32]:
score = knn.score(X_test_new,y_test)
print (score)

0.965044679613


In [33]:
k_range = list(range(1,31))
k_scores = []
for k in k_range:
    knn=KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_new,y_train)
    scores = knn.score(X_test_new,y_test)
    k_scores.append(scores)
    print (scores)
print (k_scores)

0.978332533307
0.97115137027
0.970649102116
0.966173193504
0.965044679613
0.961741789664
0.95981536877
0.95706243066
0.955482511214
0.95354337467
0.951925308275
0.950068823453
0.948504798568
0.94637174837
0.945240055567
0.943710998719
0.942169226221
0.939943987564
0.938341815731
0.936653813264
0.934698782159
0.932657920419
0.931240125503
0.929291452223
0.928080286611
0.926465399128
0.925133434847
0.923626630385
0.92278739752
0.921674778191
[0.97833253330705428, 0.97115137027017573, 0.97064910211620192, 0.96617319350357467, 0.96504467961331708, 0.9617417896640843, 0.95981536876972917, 0.95706243065997398, 0.95548251121361338, 0.95354337466979044, 0.95192530827502675, 0.9500688234527439, 0.9485047985682179, 0.9463717483700127, 0.94524005556738822, 0.9437109987188983, 0.94216922622094079, 0.93994398756409481, 0.93834181573116571, 0.93665381326432973, 0.93469878215867219, 0.93265792041910778, 0.93124012550346025, 0.92929145222253662, 0.92808028661073905, 0.92646539912834225, 0.925133434846

In [88]:
NON_FEATURES=['activity_id','people_people_id','date','people_date','outcome']
temp=test_dataset.sort_values(by='activity_id').drop(NON_FEATURES,axis=1,errors='ignore')
X_new = ipca.transform(temp.as_matrix())
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_new_t,y.as_matrix())
knn.predict_proba(X_new)

array([[ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [89]:
predictions = knn.predict_proba(X_new)

In [90]:
test_dataset.sort_values(by='activity_id',inplace=True)

In [91]:
test_dataset['outcome']=predictions[:,1]

In [92]:
test_dataset[['activity_id','outcome']].set_index(['activity_id']).to_csv('../results.csv')